In [42]:
#import libraries
import numpy as np
import pprint
import scipy
import scipy.linalg
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
import matplotlib.pyplot as pl
import json
from tqdm import tqdm
from collections import defaultdict
np.set_printoptions(threshold=20)
pd.options.mode.copy_on_write = True

In [43]:
#FUNCTION TO CONVERT CSV TO DATAFRAME
#converts and reduces dataframe to usable data
def csv_to_df(string1, string2):    
    df = pd.read_csv(string1)

    df_book = pd.read_csv(string2)

    category_array = df_book['categories'].unique()

    category_array.size
    #print('\n'.join(category_array.values))



    #Preprocessing/Data Cleaning
    df_ratings = df

    drop_columns = ["Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

    books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

    #Create Dictionary Key for Unique ID:Book
    book_dict = dict()
    for row in books_ratings_cleaned.itertuples():
        if row[1] not in book_dict:
            book_dict[row[1]] = [row[2],0]

        book_dict[row[1]][1] +=1


    drop_columns = ["Title", "Price", "profileName", "review/time", "review/summary", "review/helpfulness", "review/text"]

    books_ratings_cleaned = df_ratings.drop(drop_columns, axis=1)

    #books_ratings_cleaned

    books_ratings_cleaned['User_id'].isna()

    sum(books_ratings_cleaned['User_id'].isna())

    books_ratings_cleaned = books_ratings_cleaned[books_ratings_cleaned['User_id'].notna()]

    return books_ratings_cleaned, book_dict

In [44]:
#function to convert data frame to numpy matrix
def Convert_to_Matrix(matrix):
    #convert Dictionary Key into Sparse Matrix With users as Columns, Books as rows
    test = pd.DataFrame.from_dict(matrix)

    test.head()

    #take Matrix Transpose
    test_transpose = test.transpose()

    #populate NaN's as zeroes
    A_transpose = test_transpose.fillna(0)

    return A_transpose

#function to perform LU factorization on numpy matrix
def LU_Factorization(matrix):
    #A = PLU FACTORIZATION
    P, L, U = scipy.linalg.lu(matrix)

    A_test = np.matmul(P, np.matmul(L, U))

    #Test for Accuracy
    print(np.allclose(matrix, A_test))
    
    return P, L, U
    

In [45]:
#COLLABORATIVE FILTERING COSINE SIMILARITY FUNCTION
#finding similar users by using cosine similarity algorithm
def find_similar(user_1, k):
    allusers = A.values
    denominator1 = np.sqrt(sum([np.square(x) for x in user_1]))
    
    #performs cosine similarity algorithm on vectors (users)
    cosinesimilarity = [(user_1.name,1)]
    i=1
    for user in tqdm(allusers[1:]):
        numerator = [x*y for x,y in zip(user_1.values, user)]
        denominator2 = np.sqrt(sum([np.square(x) for x in user]))
        costheta = sum(numerator) / (denominator1 * denominator2)
        cosinesimilarity.append((A.index[i],costheta))
        i+=1
    
    #sort the results
    cosinesimilarity.sort(key = lambda x: x[1], reverse = True)
    
    #combine sorted results
    similar10users = cosinesimilarity[0:k]
    
    #output results of sorted similar users
    for i in range(0,k):
        print(similar10users[i])
    
    #place similar users into a data frame
    top10usersdf = pd.DataFrame()
    for user in similar10users:
        top10usersdf = top10usersdf.append(A.loc[user[0]])
    top10usersdf['costheta'] = [user[1] for user in similar10users]
    
    #to be used in calculation of distance/similarity for recommendation
    all_values = top10usersdf.values

    return top10usersdf

In [52]:
# Some helper methods for getting subsets of users that have more data, reducing the sparcity of matrix
# when experimenting with the dataset
def get_most_frequent_users(count, source_frame):
    assert isinstance(count, int), "Argument must be an integer"
    assert count > 0, "Argument must be a positive integer"
    
    user_occurences = source_frame['User_id'].value_counts()
    
    assert count < user_occurences.count(), "Requested more data then we have...not tiny"
    
    return user_occurences.head(count).index.to_list()

def get_users_with_minimal_ratings(rated, source_frame):
    assert isinstance(rated, int), "Argument must be an integer"
    assert rated > 0, "Argument must be a positive integer"
    
    # Get a series mapping user ids to their occurences in the data frame
    user_occurences = source_frame['User_id'].value_counts()
    return user_occurences[user_occurences > rated].index.to_list()

def get_training_data(ratio: float, source_frame):
    assert isinstance(ratio, float), "Ratio must be a float"
    assert 0.0 < ratio < 1.0, "Ratio must be in [0, 1]"
    
    # we want to ensure that our test data is built from good users, users with
    # less then 10 ratings are considered un-testable.
    good_users = get_users_with_minimal_ratings(10, source_frame)
    
    # Randomize the order of ratings for users in our good users set.
    good_users_ratings = source_frame[source_frame['User_id'].isin(good_users)].sample(frac=1)
    
    total_count = good_users_ratings.shape[0]
    training_data_count = int(total_count * ratio)
    
    training_data = pivot_rating_to_user_frame(good_users_ratings[0:training_data_count])
    test_data = pivot_rating_to_user_frame(good_users_ratings[training_data_count:])
    
    return training_data, test_data

def pivot_rating_to_user_frame(source_frame):
    new_data = dict()
    for _, row in tqdm(source_frame.iterrows(), total=source_frame.shape[0]):
        (user_id, book_id, score) = row
        if user_id not in new_data:
            new_data[user_id] = defaultdict(lambda: np.nan)
        new_data[user_id][book_id] = score
        
    return pd.DataFrame.from_dict(new_data)

In [47]:
book_ratings_cleaned, book_dict = csv_to_df("Books_rating.csv", "books_data.csv")

In [53]:
#EXCUTE ME PLEASE FOR REAL IM DYING
#LIKE REALLY, RUN ME
training_data, test_data = get_training_data(0.75, books_ratings_cleaned.sample(500000))

100%|█████████████████████████████████████████████████████████████████████████| 18185/18185 [00:01<00:00, 12134.29it/s]


In [49]:
A = training_data.fillna(0)

A.head()

,0394750594,0393014894,B000MVVEHU,1586214977,B000GSH3S8,0049220292,B0006EAC2Q,0972541802,B000J36YJE,1576463443,...,B0006AI088,1576830691,B000NSDOES,B0007DNE2G,B000GQJP2M,B000O2TIDY,B000NPVN5I,140135923X,0439224063,0393310086
A30AO422P01XK0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2AYSFGUP5VTY3,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A21X0EO0R0T6FT,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1S3C5OFU508P3,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1N1YEMTI9DJ86,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
A_similar = find_similar(A.iloc[0],10)

A_similar

100%|████████████████████████████████████████████████████████████████████████████████| 556/556 [00:23<00:00, 23.67it/s]
C:\Users\14159\AppData\Local\Temp\ipykernel_1820\1443213183.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])
C:\Users\14159\AppData\Local\Temp\ipykernel_1820\1443213183.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])
C:\Users\14159\AppData\Local\Temp\ipykernel_1820\1443213183.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])
C:\Users\14159\AppData\Local\Temp\ipykernel_1820\1443213183.py:30: FutureWarning: The frame.append method is deprecated a

('A30AO422P01XK0', 1)
('AXVXIWJ6IU6KA', 0.12987537942494304)
('A3T0OTH5072YRE', 0.12192568589855471)
('A36WD8GXQ5QCJ7', 0.0898248327232511)
('AW7BIYHXUIZ62', 0.07644509315459483)
('A1ARA52HB238HT', 0.07388543206857695)
('A1CHM200OEN65X', 0.07137902584175256)
('A1LVZOK9F7K4CN', 0.0700082424432373)
('A3CWEK9HY2VPDL', 0.06916610887771518)
('A3J0OXB9KIC5SS', 0.06313973007863823)


C:\Users\14159\AppData\Local\Temp\ipykernel_1820\1443213183.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])
C:\Users\14159\AppData\Local\Temp\ipykernel_1820\1443213183.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top10usersdf = top10usersdf.append(A.loc[user[0]])


,0394750594,0393014894,B000MVVEHU,1586214977,B000GSH3S8,0049220292,B0006EAC2Q,0972541802,B000J36YJE,1576463443,...,1576830691,B000NSDOES,B0007DNE2G,B000GQJP2M,B000O2TIDY,B000NPVN5I,140135923X,0439224063,0393310086,costheta
A30AO422P01XK0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
AXVXIWJ6IU6KA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.129875
A3T0OTH5072YRE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.121926
A36WD8GXQ5QCJ7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089825
AW7BIYHXUIZ62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076445
A1ARA52HB238HT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.073885
A1CHM200OEN65X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071379
A1LVZOK9F7K4CN,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070008
A3CWEK9HY2VPDL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069166
A3J0OXB9KIC5SS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063140


In [51]:
#USE THIS TO CHECK BOOK ID
#print(book_dict.get('1557424470'))